In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tqdm import tqdm
from Bio.Align import substitution_matrices
import pyensembl
from scipy.optimize import curve_fit

from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np
from keras.applications import ResNet101, NASNetLarge
import matplotlib.pyplot as plt
from keras import models
from keras import layers
import tensorflow as tf
import keras
from keras import layers
from keras import Input
from keras.models import Model

import time
from keras import backend as K
import os
from PIL import Image
from tqdm import tqdm, tqdm_notebook
import tensorflow as tf
from tensorflow.keras.metrics import AUC

In [ ]:
data = pd.read_csv('data_for_model_2_ds.csv')
data.head()

In [ ]:
data_1 = data[data['target']==1]
data_0 = data[data['target']==0]

In [ ]:
data_0 = data_0.sample(frac=len(data_1)/len(data_0))

In [ ]:
data = data_1.append(data_0)

In [ ]:
structures = pd.read_csv('structures_all.csv')

In [ ]:
structures['pair'] = structures['pair'].astype('int').astype('str')

In [ ]:
structures['max_prob'] = structures['max_prob'].apply(lambda p: str(np.round(p, 1)))

In [ ]:
struc_seq = []
for ENST in pd.unique(structures['ENST']):
    struc = structures[structures['ENST']==ENST]
    struc = struc.sort_values(by=['ind'])
    pair = ''.join(struc['pair'])
    prob = ';'.join(struc['max_prob'])
    struc_seq.append([ENST, pair, prob])
#     break

In [ ]:
struc_seq = pd.DataFrame(struc_seq, columns=('ENST', 'pair', 'prob'))

In [ ]:
data = pd.merge(data, struc_seq, on=['ENST'])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(data, data['target'], test_size=0.2, random_state=42, stratify=data['target'])

In [ ]:
dict_n = {'A':0, 'T':1, 'C':2, 'G':3}

In [ ]:
batch_size = 16

In [ ]:
i = 0
image1 = np.zeros((batch_size, 25, 4))
image2 = np.zeros((batch_size, 10000, 6))
target = np.zeros((batch_size, 1))
for k in range(batch_size):
    example = X_train.iloc[i]
    
    lncrna = example['lncrna_read']
    mirna = example['mirna_read']
    
    pair = example['pair']
    prob = example['prob'].split(';')
    
    change = {'A':'T', 'T':'A', 'C':'G', 'G':'C', '_':'_'}

    mirna = mirna.replace('U', 'T')
    mirna = mirna[::-1]
    mirna = ''.join([change[i] for i in mirna])
    lncrna = lncrna.replace('U', 'T')

    if len(lncrna)<=10000:
        for j in range(len(mirna)):
            Nucl = dict_n[mirna[j]]
            image1[k, j, Nucl] = 1

        for j in range(len(lncrna)):
            Nucl = dict_n[lncrna[j]]
            image2[k, j, Nucl] = 1
            image2[k, j, 4] = int(pair[j])
            image2[k, j, 5] = float(prob[j])
            
            
    target[k] = example['target']
    i += 1

In [ ]:
def generator_train(i=0, batch_size = batch_size):
    while True:
        image1 = np.zeros((batch_size, 25, 4))
        image2 = np.zeros((batch_size, 10000, 4))
        target = np.zeros((batch_size, 1))
        for k in range(batch_size):
            example = X_train.iloc[i]

            lncrna = example['lncrna_read']
            mirna = example['mirna_read']

            pair = example['pair']
            prob = example['prob'].split(';')

            change = {'A':'T', 'T':'A', 'C':'G', 'G':'C', '_':'_'}

            mirna = mirna.replace('U', 'T')
            mirna = mirna[::-1]
            mirna = ''.join([change[i] for i in mirna])
            lncrna = lncrna.replace('U', 'T')

            if len(lncrna)<=10000:
                for j in range(len(mirna)):
                    Nucl = dict_n[mirna[j]]
                    image1[k, j, Nucl] = 1

                for j in range(len(lncrna)):
                    Nucl = dict_n[lncrna[j]]
                    image2[k, j, Nucl] = 1
#                     image2[k, j, 4] = int(pair[j])
#                     image2[k, j, 5] = float(prob[j])


            target[k] = example['target']
            i += 1
        if i>=len(X_train)-batch_size+1:
            i = 0
        yield  [image1, image2], target

In [ ]:
def generator_test(i=0, batch_size = batch_size):
    while True:
        image1 = np.zeros((batch_size, 25, 4))
        image2 = np.zeros((batch_size, 10000, 4))
        target = np.zeros((batch_size, 1))
        for k in range(batch_size):
            example = X_test.iloc[i]

            lncrna = example['lncrna_read']
            mirna = example['mirna_read']

            pair = example['pair']
            prob = example['prob'].split(';')

            change = {'A':'T', 'T':'A', 'C':'G', 'G':'C', '_':'_'}

            mirna = mirna.replace('U', 'T')
            mirna = mirna[::-1]
            mirna = ''.join([change[i] for i in mirna])
            lncrna = lncrna.replace('U', 'T')

            if len(lncrna)<=10000:
                for j in range(len(mirna)):
                    Nucl = dict_n[mirna[j]]
                    image1[k, j, Nucl] = 1

                for j in range(len(lncrna)):
                    Nucl = dict_n[lncrna[j]]
                    image2[k, j, Nucl] = 1
#                     image2[k, j, 4] = int(pair[j])
#                     image2[k, j, 5] = float(prob[j])


            target[k] = example['target']
            i += 1
        if i>=len(X_test)-batch_size+1:
            i = 0
        yield  [image1, image2], target

In [ ]:
k = 0.5
tf.keras.backend.clear_session()
input_x = Input(shape=(25, 4))
x = layers.Conv1D(int(2048*k), (8), activation='relu')(input_x)
x = layers.MaxPooling1D((2))(x)
x = layers.Dropout(0.05)(x)
# x = layers.LSTM(8, return_sequences=True)(x)
# x = layers.Conv1D(1024, (4), activation='relu', padding='causal')(x)
x = layers.Conv1D(int(1024*k), (4), activation='relu')(x)
# x = layers.MaxPooling1D((2))(x)
x = layers.Dropout(0.05)(x)
x = layers.Conv1D(int(512*k), (4), activation='relu')(x)
# x = layers.MaxPooling1D((2))(x)
x = layers.Dropout(0.05)(x)
# x = layers.GlobalAveragePooling1D()(x)
x = layers.Flatten()(x)

input_y = Input(shape=(10000, 4))
y = layers.Conv1D(int(2048*k), (8), activation='relu')(input_y)
y = layers.MaxPooling1D((2))(y)
y = layers.Dropout(0.05)(y)
# y = layers.LSTM(8, return_sequences=True)(y)
y = layers.Conv1D(int(1024*k), (4), activation='relu')(y)
y = layers.MaxPooling1D((2))(y)
y = layers.Dropout(0.05)(y)
y = layers.Conv1D(int(512*k), (4), activation='relu')(y)
y = layers.MaxPooling1D((2))(y)
y = layers.Dropout(0.05)(y)
y = layers.Flatten()(y)




concatenated = layers.concatenate([x, y], axis=-1)
out = layers.Dense(1, activation='sigmoid')(concatenated)
model = Model([input_x, input_y], out)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-5), metrics=['acc'])

In [ ]:
train_size = X_train.shape[0]//batch_size
test_size = X_test.shape[0]//batch_size

In [ ]:
callbacks_list=[
                keras.callbacks.EarlyStopping(monitor='val_acc', patience=1500),
                keras.callbacks.ModelCheckpoint(filepath='best_model_2ds.h5', monitor='val_acc', 
                                                save_best_only=True, mode='max')
                ]

In [ ]:
history = model.fit_generator(
                generator_train(),
                steps_per_epoch=train_size,
                epochs=1000,
                validation_data=generator_test(),
                validation_steps=test_size,
                callbacks=callbacks_list
            )

In [ ]:
model = models.load_model('best_model_2ds.h5')

In [ ]:
pred_all = []
y_all = []
k = 0
for i in generator_test():
    pred = model.predict(i[0])
    y = i[1]
    pred_all = pred_all + list(pred)
    y_all = y_all + list(y)
    k = k + 1
    if k>test_size:
        break

In [ ]:
pred_all = np.array([i[0] for i in pred_all])

In [ ]:
pred_all2 = np.where(pred_all>0.9, 1, 0)

In [ ]:
confusion_matrix(y_all, pred_all2)

In [ ]:
print(classification_report(y_all, pred_all2))